In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import (Dense, CuDNNLSTM, Dropout, Conv1D, Conv2D, Reshape, Activation, MaxPooling2D, Flatten,
                        BatchNormalization)
import os
import tensorflow as tf
import keras
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.losses import huber_loss
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import StandardScaler
import seaborn


Using TensorFlow backend.
/opt/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519

In [2]:
# 给ret分类
def label_ret(ret):
    '''
    class 0 = [-inf, -0.1]
    class 1 = [-0.1, 0.1] unprofitable
    class 2 = [0.1, inf]
    '''
    label = None
    if ret < -0.1:
        label = 0
    elif -0.1 <= ret and ret <= 0.1:
        label = 1
    else:
        label = 2
        
    return label
def label_ret_bi(ret):
    label = None
    if ret <= 0:
        label = 0
    else:
        label = 1
    return label

def label_ret2(ret):

    if -0.1 <= ret and ret <= 0.1:
        return 0
    elif 0.1 < ret and ret <= 0.3:
        return 1
    elif 0.3 < ret:
        return 2
    elif -0.3 <= ret and ret < -0.1:
        return 3
    elif ret < -0.3:
        return 4
    else:
        raise ValueError

        
def generate_sequence(X_df, y_series, seq_length):
    assert (X_df.index == y_series.index).all()
    dataX = list()
    dataY = list()
    index = list()
    for i in range(0, X_df.shape[0] - seq_length + 1):
        dataX.append(X_df[i:i+seq_length])
        dataY.append(y_series[i+seq_length-1])
        index.append(y_series.index[i+seq_length-1])
        
    return dataX, dataY, pd.Index(index)

In [3]:
variety = 'RB'
factor_store = pd.HDFStore('/home/data/vb/training_x_150.h5', mode='r')
factor_df = factor_store.get(variety)
y_store = pd.HDFStore('/home/data/vb/training_y_reg_150.h5', mode='r')
y_series = y_store.get(variety)
helper_df = pd.read_parquet('/home/data/vb/training_helper_150_{}.parquet'.format(variety))

# 对ret做分类
ret_y_series = np.exp(y_series) - 1 # 获得回报的原始收益

label_y_series = ret_y_series.transform(label_ret_bi).rename('Y_label') # 分类标签
ret_label_df = pd.concat([ret_y_series, label_y_series], axis=1) # 合并log ret和label
assert (factor_df.index == label_y_series.index).all() # 确认数据和标签索引一样


# 对齐日期 去掉na
pd.options.mode.use_inf_as_na = True
df = helper_df.join(factor_df, how='inner').join(label_y_series, how='inner')
df.dropna(inplace=True)

# 再次得到 factor_df, label_y_series, helper_df
factor_df = df[factor_df.columns]
label_y_series = df[label_y_series.name]
helper_df = df[helper_df.columns]

assert (factor_df.index == label_y_series.index).all() and \
        (label_y_series.index == helper_df.index).all()     # 确认数据和标签索引一样


# normalize data 在这里会丢失dataframe
scaler = StandardScaler()
scaler.fit(factor_df)
factor_normalized = scaler.transform(factor_df)



# 将dataframe的index和columns加回去
factor_df_normalized = pd.DataFrame(factor_normalized, 
                                          index=factor_df.index, columns=factor_df.columns)

del factor_normalized




In [4]:
# 给lstm制造时间序列数据

SEQ_LENGTH = 100
X, y, index = generate_sequence(factor_df_normalized, label_y_series, SEQ_LENGTH)


X = np.array([factor_seq_df.values for factor_seq_df in X]) # 将list 转换为ndarray

y_cat = keras.utils.to_categorical(y) # 标签转换为one hot


In [5]:
def set_gpu_option():
    os.environ["CUDA_VISIBLE_DEVICES"]="1" 
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    keras.backend.tensorflow_backend.set_session(sess)
    
    return sess
    
def create_lstm_model(input_shape, num_classes):
    NUM_NEURONS = 1
    MULTIPLIER = 1
    model = Sequential()
    model.add(CuDNNLSTM(64*MULTIPLIER, input_shape=input_shape, return_sequences=True))
    model.add(CuDNNLSTM(32*MULTIPLIER, return_sequences=False))
    model.add(Dense(16*MULTIPLIER, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  
    model.summary()
    
    return model

def create_lstm_model_dropout(input_shape, num_classes):
    NUM_NEURONS = 1
    MULTIPLIER = 8
    model = Sequential()
    model.add(CuDNNLSTM(64*MULTIPLIER, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(CuDNNLSTM(32*MULTIPLIER, return_sequences=False))
    model.add(Dense(16*MULTIPLIER, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  
    model.summary()
    
    return model

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Reshape(input_shape+(1,), input_shape=input_shape))
    model.add(Conv2D(32, (5, 5), padding='same', activation='relu'))

    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.summary()
    
    return model

def create_cnn_conv1d_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=20, kernel_size=30, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=30, kernel_size=30, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=30, kernel_size=30, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    
    return model

In [6]:
from keras import backend as K
def Precision(y_true, y_pred):
    """精确率"""
    tp= K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))  # true positives
    pp= K.sum(K.round(K.clip(y_pred, 0, 1))) # predicted positives
    precision = tp/ (pp+ K.epsilon())
    return precision
    
def Recall(y_true, y_pred):
    """召回率"""
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # true positives
    pp = K.sum(K.round(K.clip(y_true, 0, 1))) # possible positives
    recall = tp / (pp + K.epsilon())
    return recall
 
def F1(y_true, y_pred):
    """F1-score"""
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1    


In [7]:
NUM_CLASSES = y_cat.shape[1]
sess = set_gpu_option()
input_shape = (X.shape[1], X.shape[2])
print('input_shape: ', input_shape)
model = create_cnn_conv1d_model(input_shape=input_shape ,num_classes=NUM_CLASSES)
adam = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), F1])

input_shape:  (100, 176)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 71, 20)            105620    
_________________________________________________________________
batch_normalization_1 (Batch (None, 71, 20)            80        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 42, 30)            18030     
_________________________________________________________________
batch_normalization_2 (Batch (None, 42, 30)            120       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 13, 30)            27030     
_________________________________________________________________
batch_normalization_3 (Batch (None, 13, 30)            120       
_________________________________________________________________
flatten_1 (Flatten)          

In [18]:
model.fit(x=X[-20000:-1000], y=y_cat[-20000:-1000], epochs=20, verbose=True, 
              batch_size=None, validation_data=(X[-1000:], y_cat[-1000:]), 
              shuffle=False)

Train on 19000 samples, validate on 1000 samples
Epoch 1/20
19000/19000 [==============================] - 7s 394us/step - loss: 0.2513 - accuracy: 0.8941 - precision_1: 0.8941 - recall_1: 0.8941 - F1: 0.8940 - val_loss: 5.5160 - val_accuracy: 0.5400 - val_precision_1: 0.5400 - val_recall_1: 0.5400 - val_F1: 0.5273
Epoch 2/20
19000/19000 [==============================] - 7s 390us/step - loss: 0.2480 - accuracy: 0.8944 - precision_1: 0.8944 - recall_1: 0.8944 - F1: 0.8944 - val_loss: 5.9254 - val_accuracy: 0.5440 - val_precision_1: 0.5440 - val_recall_1: 0.5440 - val_F1: 0.5312
Epoch 3/20
19000/19000 [==============================] - 7s 390us/step - loss: 0.2358 - accuracy: 0.8982 - precision_1: 0.8982 - recall_1: 0.8982 - F1: 0.8982 - val_loss: 8.0457 - val_accuracy: 0.5620 - val_precision_1: 0.5620 - val_recall_1: 0.5620 - val_F1: 0.5488
Epoch 4/20
19000/19000 [==============================] - 8s 395us/step - loss: 0.2393 - accuracy: 0.8985 - precision_1: 0.8985 - recall_1: 0.8985 

In [ ]:
model.predict()

In [19]:
np.rint(model.predict(X[-100:]))

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.

In [20]:
y_cat[-1000:].sum(axis=0)/y_cat[-1000:].sum()

array([0.558, 0.442], dtype=float32)